In [1]:
#!/usr/bin/env python

import math
import vtk
import sys

mixedVis = False
drawContour = True
changeGradient = True
legActor = vtk.vtkActor()
legValActor = vtk.vtkActor()

In [2]:
def getIsoContourActor(fileNumber, divisions) :
    # Create data 
    nColumns = 0
    nRows = 0
    
    minX = 0
    maxX = 0
    
    minY = 0
    maxY = 0
    
    minValue = 0
    maxValue = 0
    
    sGrid = vtk.vtkStructuredGrid()
    scalars = vtk.vtkDoubleArray()
    points = vtk.vtkPoints()
    
    prefix = ""
    if (int(fileNumber) < 10) :
        prefix = "0"
    with open("data/geopotential/geo" + prefix + str(fileNumber) + ".grd") as f :
        lines = list(f)
        for i in range(1, 5) : 
            a = float(lines[i].strip().split(" ")[0])
            b = float(lines[i].strip().split(" ")[1])

            if (i == 1) :
                nColumns = int(a)
                nRows = int(b)
            if (i == 2) :
                minX = a
                maxX = b
            if (i == 3) :
                minY = a
                maxY = b
            if (i == 4) :
                minValue = a
                maxValue = b
#         sGrid.SetDimensions(nRows, nColumns, 1)
        sGrid.SetDimensions(nColumns, nRows, 1)
                
        # Add points
        pointCounter = 0
        for i in range (0, nRows) :
            for j in range (0, nColumns) :
                x = j * maxX / (nColumns - 1) 
                y = i * maxY  / (nRows - 1)
                
                points.InsertPoint(pointCounter, x, y, 0)
                pointCounter += 1
        
                
#         Add scalar values
        for i in range(5, len(list(lines))) :
            values = lines[i].strip().split(" ")
            for value in values :
                scalars.InsertNextValue(float(value))       
    sGrid.SetPoints(points)
    sGrid.GetPointData().SetScalars(scalars)
    
    # Create contour   
    scalarIsoSurface = vtk.vtkContourFilter()
    scalarIsoSurface.SetInputData(sGrid)
    scalarIsoSurface.GenerateValues(divisions, minValue, maxValue)
    scalarIsoSurface.Update()
        
    scalarIsoSurfaceMapper = vtk.vtkPolyDataMapper()
    scalarIsoSurfaceMapper.SetInputData(scalarIsoSurface.GetOutput())
    scalarIsoSurfaceMapper.Update()
    
    # Isocontour + transfer function -----------------------------------------
    global mixedVis
    
    if mixedVis :
        transferRange = scalarIsoSurface.GetOutput().GetScalarRange()
        nCellsTransfer = scalarIsoSurface.GetOutput().GetNumberOfCells()
        bounds = [0.0] * 6

        for i in range(0, nCellsTransfer) :
            if (i % 2) :
                scalarIsoSurface.GetOutput().GetCellBounds(i, bounds)

    #   Create a transfer function to color the isosurfaces
        lut = vtk.vtkColorTransferFunction()
        lut.SetColorSpaceToRGB()
        lut.AddRGBPoint(transferRange[0], 0, 1, 0) # Gradient colours
        lut.AddRGBPoint(transferRange[1], 1, 0.25, 0)
        lut.SetScaleToLinear()

        scalarIsoSurfaceMapper.SetLookupTable(lut)
        scalarIsoSurfaceMapper.SetInputConnection(scalarIsoSurface.GetOutputPort())
        scalarIsoSurfaceMapper.ScalarVisibilityOn()
        scalarIsoSurfaceMapper.SetColorModeToMapScalars()

        contourActor = vtk.vtkActor()
        contourActor.SetMapper(scalarIsoSurfaceMapper)

        return contourActor
            
    scalarIsoSurfaceActor = vtk.vtkActor()
    scalarIsoSurfaceActor.SetMapper(scalarIsoSurfaceMapper)
    scalarIsoSurfaceActor.GetProperty().SetColor(colors.GetColor3d("Red"))
    
    return scalarIsoSurfaceActor

In [3]:
def MakeCellData(scalars, lut, colors, minV, maxV):
    for i in range(0, len(scalars)):
        rgb = [0.0, 0.0, 0.0]
        
        value = (scalars[i] - minV) / (maxV - minV)
        
        lut.GetColor(value, rgb)
        ucrgb = list(map(int, [x * 255 for x in rgb]))
        colors.InsertNextTuple3(ucrgb[0], ucrgb[1], ucrgb[2])

def MakeLegendCellData(lut, colors):
    for i in range(0, 101):
        rgb = [0.0, 0.0, 0.0]
        
        value = float(i) / float(100)
        
        lut.GetColor(value, rgb)
        ucrgb = list(map(int, [x * 255 for x in rgb]))
        colors.InsertNextTuple3(ucrgb[0], ucrgb[1], ucrgb[2])

def getTransferActor(fileNumber):
    nc = vtk.vtkNamedColors()
    
    # Create data 
    nColumns = 0
    nRows = 0
    
    minX = 0
    maxX = 0
    
    minY = 0
    maxY = 0
    
    minValue = 0
    maxValue = 0
    
    scalarsArray = []
    
    prefix = ""
    if (int(fileNumber) < 10) :
        prefix = "0"
    with open("data/geopotential/geo" + prefix + str(fileNumber) + ".grd") as f :
        lines = list(f)
        for i in range(1, 5) : 
            a = float(lines[i].strip().split(" ")[0])
            b = float(lines[i].strip().split(" ")[1])

            if (i == 1) :
                nColumns = int(a)
                nRows = int(b)
            if (i == 2) :
                minX = a
                maxX = b
            if (i == 3) :
                minY = a
                maxY = b
            if (i == 4) :
                minValue = a
                maxValue = b

                
        for i in range(5, len(list(lines))) :
            values = lines[i].strip().split(" ")
            for value in values :  
                scalarsArray.append(float(value))  
                         
    # Provide some geometry
    plane = vtk.vtkPlaneSource()
    plane.SetXResolution(35)
    plane.SetYResolution(25)
    plane.SetOrigin(0.0, 0.0, 0.0)
    plane.SetPoint1(float(maxX), 0.0, -0.0001);
    plane.SetPoint2(0.0, float(maxY), -0.0001);

    #  Force an update so we can set cell data
    plane.Update()

    #  Get the lookup tables mapping cell data to colors
    lut = vtk.vtkColorTransferFunction()
    lut.SetColorSpaceToRGB()
    
    global changeGradient
    if (changeGradient) :
        lut.AddRGBPoint(0, 0.6, 0.8, 0.95) # Gradient colours
        lut.AddRGBPoint(1, 0.9, 0.95, 0.6) 
    else :
        lut.AddRGBPoint(0, 0.3, 0.8, 0.3) # Gradient colours
        lut.AddRGBPoint(1, 0.8, 0.45, 0.3)
    
    lut.SetScaleToLinear()

    colorData = vtk.vtkUnsignedCharArray()
    colorData.SetName('colors')  # Any name will work here.
    colorData.SetNumberOfComponents(3)
    MakeCellData(scalarsArray, lut, colorData, minValue, maxValue)
    plane.GetOutput().GetCellData().SetScalars(colorData)

    # Set up actor and mapper
    mapper = vtk.vtkPolyDataMapper()
    mapper.SetInputConnection(plane.GetOutputPort())
    mapper.SetScalarModeToUseCellData()
    mapper.Update()

    actor = vtk.vtkActor()
    actor.SetMapper(mapper)
    
    # Create legend
    hueLut = vtk.vtkLookupTable()
    hueLut.SetTableRange(int(minValue), int(maxValue))
    hueLut.Build()
    
    scalarBar = vtk.vtkScalarBarActor()
    scalarBar.SetOrientationToVertical()
    scalarBar.SetLookupTable(mapper.GetLookupTable())
    scalarBar.SetNumberOfLabels(2)
    scalarBar.SetMaximumWidthInPixels(100)
    scalarBar.SetTitle("Geopotential height")
    
    scalarBar2 = vtk.vtkScalarBarActor()
    scalarBar2.SetOrientationToVertical()
    scalarBar2.SetLookupTable(mapper.GetLookupTable())
    scalarBar2.SetNumberOfLabels(0)
    scalarBar2.SetMaximumWidthInPixels(100)
    scalarBar2.SetTitle("Geopotential height")

    scalarBar.SetLookupTable(hueLut)
    scalarBar2.SetLookupTable(lut)
    
    global legActor
    global legValActor
    renderer.RemoveActor(legActor)
    renderer.RemoveActor(legValActor)
    
    legValActor = scalarBar2
    legActor = scalarBar
    
    renderer.AddActor(legActor)
    renderer.AddActor(legValActor)
    
#     renderer.AddActor2D()
    
    return actor

In [4]:
iren = vtk.vtkRenderWindowInteractor()
renderer = vtk.vtkRenderer()
renWin = vtk.vtkRenderWindow()

colors = vtk.vtkNamedColors()

currentFile = 0
currentDivisions = 25
# isoContourActor = getIsoContourActor(currentFile, currentDivisions)

In [5]:
def getCurrentVisActor(fileNumber, divisions) :
    global drawContour
    
    if (drawContour) : 
        renderer.RemoveActor(legActor)
        renderer.RemoveActor(legValActor)
        global isoContourActor
        return getIsoContourActor(fileNumber, divisions)
    
    else :
        return getTransferActor(fileNumber)

class KeyPressedInteractorStyle(vtk.vtkInteractorStyleTrackballCamera):

    def __init__(self, parent=None):
        self.parent = iren
        self.renderMeridiansAndParallels = True
        self.AddObserver("KeyPressEvent",self.keyPressEvent)

    def keyPressEvent(self, obj, event):
        key = self.parent.GetKeySym()
        
        if key == 'q':
            sys.exit(0)
            
        if key == 'c':
            global mixedVis
            if (mixedVis) :
                mixedVis = False
            else :
                mixedVis = True
                
            renderer.RemoveActor(isoContourActor)
            isoContourActor = getCurrentVisActor(currentFile, currentDivisions)
            renderer.AddActor(isoContourActor)
            
        if key == 'z':
            global currentFile
            global isoContourActor
            if (currentFile >= 6) :
                renderer.RemoveActor(isoContourActor)
                currentFile -= 6
                isoContourActor = getCurrentVisActor(currentFile, currentDivisions)
                renderer.AddActor(isoContourActor)
            
        if key == 'x':
            global currentFile
            global isoContourActor
            if (currentFile < 48) :
                renderer.RemoveActor(isoContourActor)
                currentFile += 6
                isoContourActor = getCurrentVisActor(currentFile, currentDivisions)
                renderer.AddActor(isoContourActor)
            
        if key == 'n':
            global currentDivisions
            global isoContourActor
            if (currentDivisions > 5) :
                renderer.RemoveActor(isoContourActor)
                currentDivisions -= 1
                isoContourActor = getCurrentVisActor(currentFile, currentDivisions)
                renderer.AddActor(isoContourActor)
            
        if key == 'v':
            global currentDivisions
            global isoContourActor
            global drawContour
            
            if (drawContour) :
                drawContour = False
            else :
                drawContour = True
            
            renderer.RemoveActor(isoContourActor)
            isoContourActor = getCurrentVisActor(currentFile, currentDivisions)
            renderer.AddActor(isoContourActor)
            
        if key == 's':
            global drawContour
            if (not drawContour) : 
                global currentDivisions
                global isoContourActor
                global changeGradient

                if (changeGradient) :
                    changeGradient = False
                else :
                    changeGradient = True

                renderer.RemoveActor(isoContourActor)
                isoContourActor = getCurrentVisActor(currentFile, currentDivisions)
                renderer.AddActor(isoContourActor)
            
        if key == 'm':
            global currentDivisions
            global isoContourActor
            if (currentDivisions < 100) :
                renderer.RemoveActor(isoContourActor)
                currentDivisions += 1
                isoContourActor = getCurrentVisActor(currentFile, currentDivisions)
                renderer.AddActor(isoContourActor)
            
        if key == 'a':
            if (self.renderMeridiansAndParallels == True) : 
                renderer.RemoveActor(euroMeridiansActor)
                renderer.RemoveActor(euroParallelsActor)
                renderer.RemoveActor(euroPointsActor)
                
                self.renderMeridiansAndParallels = False
                
            else : 
                renderer.AddActor(euroMeridiansActor)
                renderer.AddActor(euroParallelsActor)
                renderer.AddActor(euroPointsActor)
                
                self.renderMeridiansAndParallels = True
                                
        renWin.Render()
        iren.Start()
            
        return

<ipython-input-5-7ff22485df10>:38: SyntaxWarning: name 'currentFile' is used prior to global declaration
  global currentFile
<ipython-input-5-7ff22485df10>:39: SyntaxWarning: name 'isoContourActor' is assigned to before global declaration
  global isoContourActor
<ipython-input-5-7ff22485df10>:47: SyntaxWarning: name 'currentFile' is assigned to before global declaration
  global currentFile
<ipython-input-5-7ff22485df10>:48: SyntaxWarning: name 'isoContourActor' is assigned to before global declaration
  global isoContourActor
<ipython-input-5-7ff22485df10>:56: SyntaxWarning: name 'currentDivisions' is used prior to global declaration
  global currentDivisions
<ipython-input-5-7ff22485df10>:57: SyntaxWarning: name 'isoContourActor' is assigned to before global declaration
  global isoContourActor
<ipython-input-5-7ff22485df10>:65: SyntaxWarning: name 'currentDivisions' is assigned to before global declaration
  global currentDivisions
<ipython-input-5-7ff22485df10>:66: SyntaxWarning:

In [6]:
def loadEuropeFile(filename, offset, points, vectors) :
    x = [0.0] * 3
    v = [0.0] * 3
 
    with open(filename) as f :
        lines = list(f)
       
        for i in range(len(lines) - 1) :
            line = lines[i]           
            if (not line.strip().isdigit()) :
                x[0] = float(line.strip().split(" ")[0])
                x[1] = float(line.strip().split(" ")[1])

                nextLine = line   

                if (not lines[i + 1].strip().isdigit()) :
                    nextLine = lines[i + 1]

                if (not filename == "data/europe/euro_points.dat") :
                    v[0] = (float(nextLine.strip().split(" ")[0]) - float(line.strip().split(" ")[0]))
                    v[1] = (float(nextLine.strip().split(" ")[1]) - float(line.strip().split(" ")[1]))
                else :
                    v[0] = 0.025
                    v[1] = 0.025

                if (x[0] <= 26.2 and x[1] <= 18.6 and x[1] >= 0.0) :
                    points.InsertPoint(offset, x)
                    vectors.InsertTuple(offset, v)

                    offset += 1
    return offset
           
def getEuropeActor(filename, colour) :
    # Create the structured grid.
    sgrid = vtk.vtkStructuredGrid()

    # We also create the points and vectors.
    vectors = vtk.vtkDoubleArray()
    vectors.SetNumberOfComponents(3)
   
    points = vtk.vtkPoints()
   
    offset = 0
    offset = loadEuropeFile(filename, offset, points, vectors)
   
    sgrid.SetPoints(points)
    sgrid.GetPointData().SetVectors(vectors)
    
    # We create a simple pipeline to display the data.
    hedgehog = vtk.vtkHedgeHog()
    hedgehog.SetInputData(sgrid)

    sgridMapper = vtk.vtkPolyDataMapper()
    sgridMapper.SetInputConnection(hedgehog.GetOutputPort())
    
    sgridActor = vtk.vtkActor()
    sgridActor.SetMapper(sgridMapper)
    sgridActor.GetProperty().SetColor(colors.GetColor3d(colour))
   
#     if (filename == "data/europe/euro_contour_nor.dat") :
    if (filename == "data/europe/euro_contour.dat") :
        sgridActor.GetProperty().SetLineWidth(2)
   
    return sgridActor

In [ ]:
# Global actors
euroMeridiansActor = getEuropeActor("data/europe/euro_meridians.dat", "Black")
euroParallelsActor = getEuropeActor("data/europe/euro_parallels.dat", "Black")
euroPointsActor = getEuropeActor("data/europe/euro_points.dat", "Black")
# euroContourActor = getEuropeActor("data/europe/euro_contour_nor.dat", "GoldenRod")
euroContourActor = getEuropeActor("data/europe/euro_contour.dat", "GoldenRod")
isoContourActor = getIsoContourActor(0, 25)

In [ ]:
def init() :
    renWin.AddRenderer(renderer)

    iren.SetInteractorStyle(KeyPressedInteractorStyle())
    iren.SetRenderWindow(renWin)    
    
    renderer.AddActor(euroMeridiansActor)
    renderer.AddActor(euroParallelsActor)
    renderer.AddActor(euroPointsActor)
    renderer.AddActor(euroContourActor)
    renderer.AddActor(isoContourActor)
   
    renderer.SetBackground(0.3, 0.3, 0.3)
    renderer.ResetCamera()

    renWin.SetSize(800, 600)

    # Interact with the data.
    renWin.Render()
    iren.Start()
    
def main():    
    init()

if __name__ == "__main__":
    main()
    

In [ ]:
%tb